<a href="https://colab.research.google.com/github/anujch144/Binary-classifier/blob/master/ErrorQuestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2
from matplotlib import patches
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv("/content/sample_data/QuestionsWithPossibleErrors - Sheet1.csv")

In [3]:
df.head()

,QuestionId,CourseId,ClassId,SubjectId,CourseChapterId,QuestionLevelId,QuestionTypeId,IsSuspended,Viewed,Attempted,Correct,TimeTaken,Accuracy,Speed,Velocity
0,554,1,10,1,67,2,1,0,26,22,9,3692,1.5054,0.1724,1.6779
1,572,1,10,1,69,1,1,0,59,48,28,7056,1.7920,0.1514,1.9434
2,586,1,9,1,6,2,1,0,27,21,8,2954,1.4637,0.2105,1.6742
3,597,1,9,3,45,2,1,0,26,26,14,2398,1.7134,0.2529,1.9662
4,622,1,10,3,103,2,1,0,171,170,136,21649,2.2255,0.1959,2.4215


In [32]:
NoOfCourses = df['CourseId'].value_counts().index.tolist()
df['result'] = 1

In [33]:
from pandas.core.indexes.base import default_index
for CourseId in NoOfCourses:
  df_course = df.loc[df['CourseId'] == CourseId]
  NoOfQuestionLevel = df_course['QuestionLevelId'].value_counts().index.tolist()
  for i in NoOfQuestionLevel:
    df_course_questionlevel  = df_course.loc[df_course['QuestionLevelId']==i]
    #Creating new DataFrame for further processing
    df_course_questionlevel_ = df_course_questionlevel[['Accuracy','Speed','Velocity']]

    #Converting into numpy
    df_course_questionlevel_ = df_course_questionlevel_.to_numpy()
    
    # Covariance matrix
    covariance  = np.cov(df_course_questionlevel_ , rowvar=False)

    # Covariance matrix power of -1
    covariance_pm1 = np.linalg.matrix_power(covariance, -1)

    # Center point
    centerpoint = np.mean(df_course_questionlevel_ , axis=0)

    # Distances between center point and 
    distances = []
    for i, val in enumerate(df_course_questionlevel_):
        p1 = val
        p2 = centerpoint
        distance = (p1-p2).T.dot(covariance_pm1).dot(p1-p2)
        distances.append(distance)

    distances = np.array(distances)

    # Cutoff (threshold) value from Chi-Sqaure Distribution for detecting outlies 
    cutoff = chi2.ppf(0.99, df_course_questionlevel_.shape[1])

    # Index of outliers
    outlierIndexes = np.where(distances > cutoff )

    # Getting QuestionId from the Index 
    df_Index = df_course_questionlevel.iloc[outlierIndexes]['QuestionId'].index.tolist()
    
    #Update result column in dataframe
    df.iloc[df_Index]=df.iloc[df_Index].assign(result = 0)

In [39]:
df['result'].value_counts()

1    87858
0      872
Name: result, dtype: int64

In [38]:
# Enter questionId to predict the result
questionId = int(input("Enter the questionId to predict: "))
if df.loc[df['QuestionId']==questionId]['result'].iloc[0]==1:
  print("No error in Question")
else:
  print("There is error in Question")  

Enter the questionId to predict: 3755
There is error in Question
